[with pytorch](https://gist.github.com/PetrochukM/afaa3613a99a8e7213d2efdd02ae4762)

[notebook](https://github.com/napsternxg/pytorch-practice/blob/master/Viterbi%20decoding%20and%20CRF.ipynb)

[with different rings](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html)

[python only?](https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm)

[numpy](http://www.adeveloperdiary.com/data-science/machine-learning/implement-viterbi-algorithm-in-hidden-markov-model-using-python-and-r/)

In [174]:
import numpy as np
import torch

np.random.seed(2017)

torch.manual_seed(2017)

# from scipy.misc import logsumexp # Use it for reference checking implementation

In [175]:
seq_length, num_states = 4, 2
emissions = np.random.randint(20, size=(seq_length, num_states))*1.
transitions = np.random.randint(10, size=(num_states, num_states))*1.

print("Emissions:", emissions, sep="\n")
print("Transitions:", transitions, sep="\n")

Emissions:
[[ 9.  6.]
 [13. 10.]
 [ 8. 18.]
 [ 3. 15.]]
Transitions:
[[7. 8.]
 [0. 8.]]


In [176]:
emissions.shape, transitions.shape

((4, 2), (2, 2))

In [177]:
def viterbi_decoding_numpy(emissions, transitions):
    # Use help from: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/crf/python/ops/crf.py
    scores = np.zeros_like(emissions)
    back_pointers = np.zeros_like(emissions, dtype="int")
    scores = emissions[0]

    # Generate most likely scores and paths for each step in sequence
    for i in range(1, emissions.shape[0]):
        score_with_transition = np.expand_dims(scores, 1) + transitions
        scores = emissions[i] + score_with_transition.max(axis=0)
        back_pointers[i] = np.argmax(score_with_transition, 0)
    

    # Generate the most likely path
    viterbi = [np.argmax(scores)]
    for bp in reversed(back_pointers[1:]):
        viterbi.append(bp[viterbi[-1]])
    viterbi.reverse()
    viterbi_score = np.max(scores)

    return viterbi_score, viterbi


viterbi_decoding_numpy(emissions, transitions)

(78.0, [0, 0, 1, 1])

In [178]:
def zeros_like(m):
    return zeros(len(m), len(m[0]))


def zeros(d1, d2):
    return list(list(0 for _ in range(d2)) for _ in range(d1))


def apply_transpose(f, m):
    return list(map(f, zip(*m)))


def argmax(l):
    return max(range(len(l)), key=lambda i: l[i])


def add1d2d(m1, m2):
    return [[v2 + v1 for v2 in m2_row] for m2_row, v1 in zip(m2, m1)]


def add1d1d(v1, v2):
    return [e + s for e, s in zip(v1, v2)]


def viterbi_decoding_list(emissions, transitions):
    scores = zeros_like(emissions)
    back_pointers = zeros_like(emissions)
    scores = emissions[0]

    # Generate most likely scores and paths for each step in sequence
    for i in range(1, len(emissions)):
        score_with_transition = add1d2d(scores, transitions)
        max_score_with_transition = apply_transpose(max, score_with_transition)
        scores = add1d1d(emissions[i], max_score_with_transition)
        back_pointers[i] = apply_transpose(argmax, score_with_transition)

    # Generate the most likely path
    viterbi = [argmax(scores)]
    for bp in reversed(back_pointers[1:]):
        viterbi.append(bp[viterbi[-1]])
    viterbi.reverse()
    viterbi_score = max(scores)

    return viterbi_score, viterbi


emissions_list = emissions.tolist()
transitions_list = transitions.tolist()

viterbi_decoding_list(emissions_list, transitions_list)

(78.0, [0, 0, 1, 1])

In [179]:
%timeit viterbi_decoding_numpy(emissions, transitions)
%timeit viterbi_decoding_list(emissions_list, transitions_list)

123 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
36 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [180]:
words = ["hello", "world", "how", "are", "you"]

In [181]:
words = ["hello", "world", "how", "are", "you", "eeeee"]
lm = "abcdefghijklmnopqrstuvwxyz"
tokens = [[lm.find(w) for w in word] for word in words]
tokens

[[7, 4, 11, 11, 14],
 [22, 14, 17, 11, 3],
 [7, 14, 22],
 [0, 17, 4],
 [24, 14, 20],
 [4, 4, 4, 4, 4]]

In [182]:
from collections import Counter


def build_transitions(data_loader, n=2):

    # Count n-grams
    count = Counter()
    for label in data_loader:
        count += Counter(a for a in zip(*(label[i:] for i in range(n))))
            
    # Write as matrix                                                                                                                                                              
    transitions = zeros(len(lm), len(lm))    
    for (k1, k2), v in count.items():
        transitions[k1][k2] = v

    return transitions


transitions = build_transitions(tokens)

In [183]:
torch.tensor(transitions)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 

In [184]:
%timeit build_transitions(tokens)

115 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
